# Challenging Limitations: Using Deep Learning, Time Series Analysis, and Statistical Method for Noise Reduction to Develop an Innovative Approach to Exoplanet Candidate Detection

## I. A Comparative Visualization of the Star Fields Visible to Earth-Based and Space-Based Telescopes

In [1]:
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import math
import ipyvolume as ipv
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink, IntSlider, ToggleButton
import ipyvolume.pylab as p3

class StarDF(object):
    def __init__(self, path, parse_placeholders, parse_prelim_nans):
        self.path = path
        self.parse_placeholders = parse_placeholders
        self.parse_prelim_nans = parse_prelim_nans
        
    def _gen_df(self):
        data = pd.read_csv(self.path)
        return data
       
    def _initialize(self):
        data = self._gen_df()
        if self.parse_placeholders is True:
            data = data.loc[data['dist'] < 100000]
            data = data.reset_index(drop=True)
        if self.parse_prelim_nans is False:
            data = data[1:]
            data = data.reset_index(drop=True)
        return data
    
    def get_full(self):
        data = self._initialize()
        return data
    
    def split(self):
        data = self._initialize()
        ra = data['ra']
        dec = data['dec']
        dist = data['dist']
        return ra, dec, dist
    
    def split_other(self, data):
        ra = data['ra']
        dec = data['dec']
        dist = data['dist']
        return ra, dec, dist
    
    def convert_to_degrees(self):
        df = self.initialize()
        ra = df['RA_STRING']
        dec = df['DEC_STRING']
        ra = ra.values
        dec = dec.values
        new_ra = np.empty(len(ra))
        new_dec = np.empty(len(dec))
        for i in range(0,len(ra)):
            coord = ra[i] + " " + dec[i]
            c = SkyCoord(coord, unit=(u.hourangle, u.deg))
            temp = c.to_string('decimal')
            temp_b = temp.split(' ')
            new_ra[i] = temp_b[0]
            new_dec[i] = temp_b[1]
        temp = {'ra': new_ra, 'dec': new_dec}
        exo_data = pd.DataFrame(temp, columns = ['ra', 'dec'])
        return exo_data
    
    def intersection(self, df1, df2, cols, full, make_non=True):
        intersect = pd.merge(df1, df2, on=cols, how='inner')
        for i in intersect['id']:
            full = full.loc[full['id'] != i]
        non = full.reset_index(drop=True)
        intersect = intersect.reset_index(drop=True)
        return intersect, non
    
    def convert_to_cartesian(self, ra,dec,dist,n =1000):
        x = np.empty(n)
        y = np.empty(n)
        z = np.empty(n)
        for i in range(0,n):
            z[i] = dist[i] * np.sin(dec[i])
            phi = dist[i] * np.cos(dec[i])
            y[i] = phi * np.sin(ra[i])
            x[i] = phi * np.cos(ra[i])
        return x,y,z
    
    def integ_cartesian(self, ra,dec,dist):
        w = dist * np.sin(dec)
        phi = dist * np.cos(dec)
        v = phi * np.sin(ra)
        u = phi * np.cos(ra)
        return u,v,w
    
    def get_stats(self, array):
        return np.mean(array), np.std(array)
    
    
    
def snip(df, length):
    new = df[:length]
    return new

#parse for dubious parallax data as indicated by HYG database
fullHYG = StarDF('stardata_hyg_v2.csv', parse_placeholders=True, parse_prelim_nans = False)
fullHYG_df = fullHYG.get_full()
fullHYG_df = snip(fullHYG_df, length= 109398)
hyg_ra, hyg_dec, hyg_dist = fullHYG.split()


exo = StarDF('exo_data.csv', parse_placeholders = False, parse_prelim_nans = True)
exo_df = exo.get_full()


hyg_temp = fullHYG_df.dropna(thresh=7)
exo_temp = exo_df.dropna(thresh=3)
cols = ['HD', 'HIPP']
exo_stars, non_exo = exo.intersection(hyg_temp, exo_temp, cols, fullHYG_df)
exo_stars = snip(exo_stars, 2951)
exo_ra, exo_dec, exo_dist = exo.split_other(exo_stars)

hyg_x,hyg_y,hyg_z = fullHYG.convert_to_cartesian(hyg_ra,hyg_dec,hyg_dist, n = len(hyg_ra))
exo_x, exo_y, exo_z = exo.convert_to_cartesian(exo_ra, exo_dec, exo_dist, n = len(exo_ra))
    
#initialize kepler vars
kep_x, kep_y, kep_z = np.empty(1), np.empty(1), np.empty(1)
kep_x[0] = 0
kep_y[0] = 0
kep_z[0] = 0
kep_u, kep_v, kep_w = np.empty(1), np.empty(1), np.empty(1)
#following params are center of kepler star field and depth of view of kepler telescope
kep_ra = 290.6667
kep_dec = 44.5
kep_dist = 919.8042
u,v,w = exo.integ_cartesian(kep_ra, kep_dec, kep_dist)
kep_u[0] = u
kep_v[0] = v
kep_w[0] = w


# 4 vertices for the tetrahedron
x_a, y_a, z_a = exo.integ_cartesian(kep_ra, kep_dec+7.5, 919.8042/np.cos(7.5))
x_b, y_b,z_b = exo.integ_cartesian(kep_ra+7.5, kep_dec, 919.8042/np.cos(7.5))
x_c, y_c, z_c = exo.integ_cartesian(kep_ra-7.5, kep_dec, 919.8042/np.cos(7.5))
x = np.array([0., x_a , x_b,  x_c])
y = np.array([0, y_a,  y_b, y_c])
z = np.array([0, z_a, z_b,  z_c])
# and 4 surfaces (triangles), where the numbers refer to the vertex index
triangles = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1,3,2)]
# we draw the tetrahedron



ipv.figure()
selected = (hyg_x[0], hyg_y[0], hyg_z[0])
scatter_a = ipv.scatter(hyg_x,hyg_y,hyg_z,size=0.15, size_selected = 0.6, marker="sphere", selected = selected)
scatter_b = ipv.scatter(hyg_x[:50000], hyg_y[:50000], hyg_z[:50000], size=0.15, marker="sphere" )
scatter_c = ipv.scatter(hyg_x[:10000], hyg_y[:10000], hyg_z[:10000], size=0.15, marker="sphere" )
scatter_d = ipv.scatter(hyg_x[:5000], hyg_y[:5000], hyg_z[:5000], size=0.15, marker="sphere" )
scatter_e = ipv.scatter(hyg_x[:3000], hyg_y[:3000], hyg_z[:3000], size=0.15, marker="sphere" )
scatter_f = ipv.scatter(hyg_x[:1000], hyg_y[:1000], hyg_z[:1000], size=0.15, marker="sphere" )
exo_scatter = ipv.scatter(exo_x, exo_y, exo_z, size = 0.6, marker = 'sphere', color= '#00c8ff')

linex =np.array([0,kep_u])
liney = np.array([0, kep_v])
linez = np.array([0,kep_w])
p3.plot(linex, liney ,linez, color='blue')
#p3.plot_trisurf(linex, liney, linez, triangles=triangles, color='orange')

size = FloatSlider(min=0.1, max=1, step=0.05, description = 'exo stars')
#color_a = ColorPicker(description = 'non-exo')
#color_selected = ColorPicker('sun')
exo_color = ColorPicker(description='exo stars')
kep_color = ColorPicker()
kep_size = FloatSlider(min=0, max = 10, step = 0.1)
toggle_all = ToggleButton(value=False,
    description='All Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
toggle_50000 = ToggleButton(value=False,
    description='50000 Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
toggle_10000 = ToggleButton(value=False,
    description='10000 Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
toggle_5000 = ToggleButton(value=False,
    description='5000 Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
toggle_3000 = ToggleButton(value=False,
    description='3000 Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
toggle_1000 = ToggleButton(value=False,
    description='1000 Stars',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
jslink((exo_scatter, 'size'), (size, 'value'))
#jslink((scatter_a, 'color'), (color, 'value'))
#jslink((scatter_a, 'color_selected'), (color_selected, 'value'))
jslink((exo_scatter, 'color'), (exo_color, 'value'))
jslink((scatter_a, 'visible'), (toggle_all, 'value'))
jslink((scatter_b, 'visible'), (toggle_50000, 'value'))
jslink((scatter_c, 'visible'), (toggle_10000, 'value'))
jslink((scatter_d, 'visible'), (toggle_5000, 'value'))
jslink((scatter_e, 'visible'), (toggle_3000, 'value'))
jslink((scatter_f, 'visible'), (toggle_1000, 'value'))
    
VBox([ipv.gcc(), size, exo_color, toggle_all, toggle_50000, toggle_10000,
     toggle_5000, toggle_3000, toggle_1000])#size, color_a, color_selected

VBox(children=(VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', selected=array([  1.01838186e+02,   6.11029114e-03,   1.94762215e+02]), size=array(0.15), size_selected=array(0.6), x=array([ 101.83818558,   38.20222153,  176.63109425, ...,    7.76314494,
         -4.8799141 ,    5.33899082]), y=array([  6.11029114e-03,   1.08112290e-02,   5.91714188e-02, ...,
        -1.67380193e+01,   1.04894062e+01,   1.92602154e-01]), z=array([ 194.76221459,  -28.99836596,  405.69464506, ...,   -1.58339327,
         -5.5248545 ,    5.68355184])), Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(0.15), size_selected=array(2.6), x=array([ 101.83818558,   38.20222153,  176.63109425, ...,  -16.15918208,
        -23.49345048,   -1.28815638]), y=array([  6.11029114e-03,   1.08112290e-02,   5.91714188e-02, ...,
         6.87563787e+01,   9.99435187e+01,   5.46946243e+00]), z=array([ 194.76221459,  -28.99836596,  405.69464506, ...,   59.47601688,
         63.88079059, -186.83141948])), Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(0.15), size_selected=array(2.6), x=array([ 101.83818558,   38.20222153,  176.63109425, ..., -212.03625375,
       -246.11140425,   34.85697033]), y=array([  6.11029114e-03,   1.08112290e-02,   5.91714188e-02, ...,
         2.21609449e+02,   2.57148398e+02,  -3.64022907e+01]), z=array([ 194.76221459,  -28.99836596,  405.69464506, ...,  388.59218555,
       -116.52445035,  -91.65995775])), Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(0.15), size_selected=array(2.6), x=array([ 101.83818558,   38.20222153,  176.63109425, ...,   30.98292621,
        213.56678181,  -87.4840481 ]), y=array([  6.11029114e-03,   1.08112290e-02,   5.91714188e-02, ...,
         7.06614483e+01,   4.87806045e+02,  -1.99953800e+02]), z=array([ 194.76221459,  -28.99836596,  405.69464506, ...,  -85.91346874,
        -74.06241054,  -25.85257517])), Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(0.15), size_selected=array(2.6), x=array([ 101.83818558,   38.20222153,  176.63109425, ...,   26.08717662,
        -31.53639706,  236.98313254]), y=array([  6.11029114e-03,   1.08112290e-02,   5.91714188e-02, ...,
         2.16398101e+01,  -2.61756977e+01,   1.96752106e+02]), z=array([ 194.76221459,  -28.99836596,  405.69464506, ..., -121.96006112,
        228.91823516,  -19.68470943])), Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(0.15), size_selected=array(2.6), x=array([  1.01838186e+02,   3.82022215e+01,   1.76631094e+02,
        -7.68267318e+00,  -2.49751373e+02,  -3.81512433e+01,
         2.16928718e+01,   1.46415063e+02,   1.85644213e+02,
         7.61300946e+01,  -2.35200016e+02,  -5.14767561e+01,
         1.92537707e+02,   4.52861631e+02,  -1.08744751e+02,
        -4.77183255e+01,  -2.74357676e+01,   2.05120252e+02,
        -3.16046290e+00,  -2.89112983e+01,   1.94648543e+02,
         6.33730283e+01,  -1.15011877e+01,   7.75304141e+01,
         6.82531013e+01,  -9.65896151e+01,   1.61106523e+02,
         1.65262702e+02,  -6.52204978e+01,  -2.74018934e+02,
        -3.40009437e+01,  -5.78238006e+01,  -1.58897873e+01,
        -5.02912253e+01,   1.76298359e+02,  -3.81078564e+02,
        -3.66631546e+01,  -5.08299524e+01,  -2.28238829e+02,
         1.23695217e+02,  -1.35074916e+02,  -3.31014788e+02,
        -6.22122831e+01,   7.35426603e+02,   3.90822015e+01,
        -6.63554612e+02,  -2.15265175e+02,  -5.65944494e+01,
        -1.07067740e+01,   3.19587685e+01,  -4.92531054e+01,
         2.47511693e+02,   2.52900219e+01,   3.07494152e+01,
        -1.08611545e+01,  -1.63689570e+02,   4.01161343e+